In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
import asyncio
import aiohttp
from datetime import datetime
import hashlib
import pickle
import os

class ResearchRAGAgent:
    def __init__(self):
        # Initialize components
        self.knowledge_base = {}
        self.research_cache = {}
        self.task_queue = []
        self.results = {}

        # Initialize widgets
        self.setup_ui()

    def setup_ui(self):
        """Setup the user interface"""
        # Main query input
        self.research_query = widgets.Textarea(
            placeholder="Enter your complex research query...",
            description='Research:',
            layout=widgets.Layout(width='95%', height='100px')
        )

        # Research depth selector
        self.depth_selector = widgets.Dropdown(
            options=[
                ('Quick Research', 'quick'),
                ('Standard Research', 'standard'),
                ('Deep Research', 'deep'),
                ('Comprehensive Analysis', 'comprehensive')
            ],
            value='standard',
            description='Depth:'
        )

        # Task delegation options
        self.delegate_tasks = widgets.Checkbox(
            value=True,
            description='Enable Task Delegation',
            tooltip='Break down complex queries into sub-tasks'
        )

        # Sources selector
        self.sources_selector = widgets.SelectMultiple(
            options=['Web Search', 'Academic Papers', 'News', 'Technical Docs', 'Forums'],
            value=['Web Search', 'News'],
            description='Sources:',
            layout=widgets.Layout(width='200px', height='100px')
        )

        # Format selector
        self.format_selector = widgets.Dropdown(
            options=['Research Paper', 'Executive Summary', 'Technical Report', 'Presentation', 'Raw Data'],
            value='Executive Summary',
            description='Format:'
        )

        # Action buttons
        self.analyze_btn = widgets.Button(
            description="Analyze Query",
            button_style='info',
            icon='search'
        )
        self.analyze_btn.on_click(self.analyze_query)

        self.research_btn = widgets.Button(
            description="Start Research",
            button_style='success',
            icon='rocket'
        )
        self.research_btn.on_click(self.start_research)

        self.clear_btn = widgets.Button(
            description="Clear All",
            button_style='warning'
        )
        self.clear_btn.on_click(self.clear_all)

        # Output areas
        self.analysis_output = widgets.Output()
        self.research_output = widgets.Output()
        self.progress_output = widgets.Output()

        # Assemble UI
        self.research_panel = widgets.VBox([
            widgets.HTML("<h1>🔬 Advanced RAG Research Agent</h1>"),
            widgets.HTML("<p>Complex research with task delegation and multi-source analysis</p>"),
            self.research_query,
            widgets.HBox([
                self.depth_selector,
                self.format_selector,
                self.delegate_tasks
            ]),
            widgets.HBox([
                self.sources_selector,
                widgets.VBox([self.analyze_btn, self.research_btn, self.clear_btn])
            ]),
            widgets.HTML("<h3>Query Analysis:</h3>"),
            self.analysis_output,
            widgets.HTML("<h3>Research Progress:</h3>"),
            self.progress_output,
            widgets.HTML("<h3>Research Results:</h3>"),
            self.research_output
        ])

    def analyze_query(self, b):
        """Analyze complex query and break down into sub-tasks"""
        self.analysis_output.clear_output()

        query = self.research_query.value.strip()
        if not query:
            with self.analysis_output:
                display(HTML("<div style='color: red;'>Please enter a research query</div>"))
            return

        with self.analysis_output:
            display(HTML(f"<h4>🔍 Analyzing: '{query}'</h4>"))

            # Analyze query complexity
            complexity = self.assess_query_complexity(query)
            display(HTML(f"<p><b>Complexity Level:</b> {complexity}</p>"))

            # Break down into sub-tasks
            sub_tasks = self.decompose_query(query)

            display(HTML("<h4>📋 Identified Sub-Tasks:</h4>"))
            for i, task in enumerate(sub_tasks, 1):
                display(HTML(f"<p>{i}. {task['task']} <em>({task['type']})</em></p>"))

            # Store tasks for research
            self.task_queue = sub_tasks
            self.results['query_analysis'] = {
                'original_query': query,
                'complexity': complexity,
                'sub_tasks': sub_tasks,
                'timestamp': datetime.now().isoformat()
            }

    def assess_query_complexity(self, query):
        """Assess the complexity of the research query"""
        words = len(query.split())

        complexity_indicators = [
            'compare', 'analyze', 'evaluate', 'comprehensive', 'deep dive',
            'multiple perspectives', 'impact assessment', 'future trends'
        ]

        indicator_count = sum(1 for indicator in complexity_indicators if indicator in query.lower())

        if words > 30 or indicator_count >= 3:
            return "High Complexity"
        elif words > 15 or indicator_count >= 1:
            return "Medium Complexity"
        else:
            return "Low Complexity"

    def decompose_query(self, query):
        """Break down complex query into manageable sub-tasks"""
        query_lower = query.lower()
        tasks = []

        # Define task patterns
        task_patterns = {
            'definition': ['what is', 'define', 'explain the concept'],
            'comparison': ['compare', 'vs', 'versus', 'difference between'],
            'analysis': ['analyze', 'examine', 'investigate'],
            'trends': ['trends', 'future', 'emerging', 'recent developments'],
            'impact': ['impact', 'effect', 'consequences', 'implications'],
            'solutions': ['solutions', 'approaches', 'methods', 'strategies']
        }

        # Add tasks based on query content
        if any(pattern in query_lower for pattern in task_patterns['definition']):
            tasks.append({
                'task': f"Define and explain core concepts in: {query}",
                'type': 'Definition',
                'priority': 'high'
            })

        if any(pattern in query_lower for pattern in task_patterns['comparison']):
            tasks.append({
                'task': f"Compare key aspects and differences",
                'type': 'Comparison',
                'priority': 'medium'
            })

        if any(pattern in query_lower for pattern in task_patterns['analysis']):
            tasks.append({
                'task': f"Conduct detailed analysis of components",
                'type': 'Analysis',
                'priority': 'high'
            })

        if any(pattern in query_lower for pattern in task_patterns['trends']):
            tasks.append({
                'task': f"Identify current and future trends",
                'type': 'Trend Analysis',
                'priority': 'medium'
            })

        if any(pattern in query_lower for pattern in task_patterns['impact']):
            tasks.append({
                'task': f"Assess impacts and implications",
                'type': 'Impact Assessment',
                'priority': 'medium'
            })

        if any(pattern in query_lower for pattern in task_patterns['solutions']):
            tasks.append({
                'task': f"Explore solutions and recommendations",
                'type': 'Solution Finding',
                'priority': 'medium'
            })

        # Default task if no specific patterns matched
        if not tasks:
            tasks.append({
                'task': f"Comprehensive research on: {query}",
                'type': 'General Research',
                'priority': 'high'
            })

        return tasks

    async def start_research(self, b):
        """Start the research process with task delegation"""
        self.research_output.clear_output()
        self.progress_output.clear_output()

        if not self.task_queue:
            with self.research_output:
                display(HTML("<div style='color: red;'>Please analyze the query first</div>"))
            return

        with self.progress_output:
            display(HTML("<h4>🚀 Starting Research Process...</h4>"))

            # Execute research tasks
            research_results = await self.execute_research_tasks()

            # Synthesize results
            final_report = await self.synthesize_results(research_results)

            # Display final report
            with self.research_output:
                self.display_final_report(final_report)

    async def execute_research_tasks(self):
        """Execute all research tasks asynchronously"""
        research_results = {}

        with self.progress_output:
            for i, task in enumerate(self.task_queue, 1):
                display(HTML(f"<p>📊 Executing Task {i}: {task['task']}</p>"))

                # Simulate research for each task
                task_result = await self.research_task(task)
                research_results[f"task_{i}"] = {
                    'task_info': task,
                    'results': task_result,
                    'sources': await self.gather_sources(task),
                    'timestamp': datetime.now().isoformat()
                }

                display(HTML(f"<p style='color: green;'>✅ Completed Task {i}</p>"))

        return research_results

    async def research_task(self, task):
        """Research a specific task"""
        # This would integrate with actual research APIs
        # For now, simulating with mock data

        await asyncio.sleep(1)  # Simulate research time

        mock_responses = {
            'Definition': f"Comprehensive definition and explanation of key concepts related to: {task['task']}",
            'Comparison': "Detailed comparison with pros/cons, use cases, and performance metrics",
            'Analysis': "In-depth analysis covering technical aspects, market trends, and implementation considerations",
            'Trend Analysis': "Current trends, emerging technologies, and future predictions with supporting data",
            'Impact Assessment': "Assessment of economic, social, and technological impacts with case studies",
            'Solution Finding': "Multiple solution approaches with implementation guidelines and best practices",
            'General Research': "Comprehensive research covering all relevant aspects with data-driven insights"
        }

        return mock_responses.get(task['type'], "Research completed with detailed findings")

    async def gather_sources(self, task):
        """Gather sources for the research task"""
        sources = []

        selected_sources = self.sources_selector.value
        if 'Web Search' in selected_sources:
            sources.extend(await self.web_search(task['task']))
        if 'Academic Papers' in selected_sources:
            sources.extend(await self.academic_search(task['task']))
        if 'News' in selected_sources:
            sources.extend(await self.news_search(task['task']))

        return sources

    async def web_search(self, query):
        """Simulate web search"""
        return [
            {"type": "web", "title": f"Research on {query}", "url": "https://example.com/research", "snippet": "Comprehensive information..."},
            {"type": "web", "title": f"Analysis: {query}", "url": "https://example.com/analysis", "snippet": "Detailed analysis..."}
        ]

    async def academic_search(self, query):
        """Simulate academic search"""
        return [
            {"type": "academic", "title": f"Academic Study: {query}", "authors": "Researcher et al.", "year": "2024"},
            {"type": "academic", "title": f"Journal Paper: {query}", "authors": "Scholar et al.", "year": "2023"}
        ]

    async def news_search(self, query):
        """Simulate news search"""
        return [
            {"type": "news", "title": f"Recent Developments: {query}", "source": "Tech News", "date": "2024"},
            {"type": "news", "title": f"Industry Update: {query}", "source": "Business Daily", "date": "2024"}
        ]

    async def synthesize_results(self, research_results):
        """Synthesize all research results into a coherent report"""
        with self.progress_output:
            display(HTML("<h4>🧠 Synthesizing Research Results...</h4>"))

        # Generate comprehensive report based on format
        report_format = self.format_selector.value

        synthesis = {
            'executive_summary': await self.generate_executive_summary(research_results),
            'key_findings': await self.extract_key_findings(research_results),
            'methodology': self.get_research_methodology(),
            'recommendations': await self.generate_recommendations(research_results),
            'sources': await self.consolidate_sources(research_results),
            'timestamp': datetime.now().isoformat()
        }

        return synthesis

    async def generate_executive_summary(self, results):
        """Generate executive summary"""
        return f"Comprehensive research analysis covering {len(results)} key areas. Key insights include..."

    async def extract_key_findings(self, results):
        """Extract key findings from research"""
        findings = []
        for task_id, task_data in results.items():
            findings.append({
                'area': task_data['task_info']['type'],
                'key_insights': [f"Important finding from {task_data['task_info']['type']} research"],
                'evidence': task_data['sources'][:2]  # Top 2 sources
            })
        return findings

    def get_research_methodology(self):
        """Document research methodology"""
        return {
            'depth': self.depth_selector.value,
            'sources': list(self.sources_selector.value),
            'task_delegation': self.delegate_tasks.value,
            'analysis_framework': 'Multi-dimensional research with task-specific approaches'
        }

    async def generate_recommendations(self, results):
        """Generate actionable recommendations"""
        return [
            "Implement strategic approach based on research findings",
            "Monitor emerging trends in key areas",
            "Consider additional research for specific sub-domains"
        ]

    async def consolidate_sources(self, results):
        """Consolidate all sources"""
        all_sources = []
        for task_data in results.values():
            all_sources.extend(task_data['sources'])
        return all_sources

    def display_final_report(self, report):
        """Display the final research report"""
        html_report = f"""
        <div style="font-family: Arial, sans-serif; line-height: 1.6;">
            <h1 style="color: #2c3e50;">🔬 Research Report</h1>
            <div style="background: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0;">
                <h2 style="color: #34495e;">Executive Summary</h2>
                <p>{report['executive_summary']}</p>
            </div>

            <h2 style="color: #34495e;">📊 Key Findings</h2>
        """

        for i, finding in enumerate(report['key_findings'], 1):
            html_report += f"""
            <div style="border-left: 4px solid #3498db; padding-left: 15px; margin: 15px 0;">
                <h3>Finding {i}: {finding['area']}</h3>
                <ul>
                    <li>{finding['key_insights'][0]}</li>
                </ul>
            </div>
            """

        html_report += f"""
            <h2 style="color: #34495e;">🎯 Recommendations</h2>
            <ul>
        """

        for rec in report['recommendations']:
            html_report += f"<li>{rec}</li>"

        html_report += f"""
            </ul>

            <h2 style="color: #34495e;">🔍 Methodology</h2>
            <p><strong>Research Depth:</strong> {report['methodology']['depth']}</p>
            <p><strong>Sources Used:</strong> {', '.join(report['methodology']['sources'])}</p>
            <p><strong>Task Delegation:</strong> {'Enabled' if report['methodology']['task_delegation'] else 'Disabled'}</p>

            <div style="background: #e8f4f8; padding: 15px; border-radius: 8px; margin-top: 20px;">
                <p><strong>Report Generated:</strong> {report['timestamp']}</p>
                <p><strong>Total Sources:</strong> {len(report['sources'])}</p>
            </div>
        </div>
        """

        display(HTML(html_report))

        # Also show structured data
        display(HTML("<h3>📁 Structured Research Data</h3>"))
        research_df = pd.DataFrame([
            {
                'Task Type': finding['area'],
                'Key Insight': finding['key_insights'][0],
                'Sources Count': len(finding['evidence'])
            }
            for finding in report['key_findings']
        ])
        display(research_df)

    def clear_all(self, b):
        """Clear all outputs and reset state"""
        self.analysis_output.clear_output()
        self.research_output.clear_output()
        self.progress_output.clear_output()
        self.task_queue = []
        self.results = {}

# Create and display the research agent
research_agent = ResearchRAGAgent()
display(research_agent.research_panel)

class TaskDelegationEngine:
    def __init__(self, research_agent):
        self.research_agent = research_agent
        self.worker_agents = {}
        self.setup_worker_agents()

    def setup_worker_agents(self):
        """Setup specialized worker agents"""
        self.worker_agents = {
            'data_analyzer': DataAnalysisAgent(),
            'trend_researcher': TrendResearchAgent(),
            'comparison_engine': ComparisonAnalysisAgent(),
            'impact_assessor': ImpactAssessmentAgent(),
            'solution_finder': SolutionFindingAgent()
        }

    async def delegate_tasks(self, main_task):
        """Delegate tasks to specialized agents"""
        delegated_results = {}

        # Analyze main task and delegate to appropriate agents
        if 'compare' in main_task.lower() or 'vs' in main_task.lower():
            delegated_results['comparison'] = await self.worker_agents['comparison_engine'].analyze(main_task)

        if 'trend' in main_task.lower() or 'future' in main_task.lower():
            delegated_results['trends'] = await self.worker_agents['trend_researcher'].research(main_task)

        if 'impact' in main_task.lower() or 'effect' in main_task.lower():
            delegated_results['impact'] = await self.worker_agents['impact_assessor'].assess(main_task)

        if 'solution' in main_task.lower() or 'approach' in main_task.lower():
            delegated_results['solutions'] = await self.worker_agents['solution_finder'].find_solutions(main_task)

        # Always include data analysis
        delegated_results['analysis'] = await self.worker_agents['data_analyzer'].analyze(main_task)

        return delegated_results

class DataAnalysisAgent:
    async def analyze(self, task):
        return {"type": "data_analysis", "results": "Comprehensive data analysis completed"}

class TrendResearchAgent:
    async def research(self, task):
        return {"type": "trend_research", "results": "Current and future trends identified"}

class ComparisonAnalysisAgent:
    async def analyze(self, task):
        return {"type": "comparison", "results": "Detailed comparison analysis completed"}

class ImpactAssessmentAgent:
    async def assess(self, task):
        return {"type": "impact_assessment", "results": "Impact analysis with metrics"}

class SolutionFindingAgent:
    async def find_solutions(self, task):
        return {"type": "solution_finding", "results": "Multiple solution approaches identified"}

class EnhancedResearchAgent(ResearchRAGAgent):
    def __init__(self):
        super().__init__()
        self.task_delegation_engine = TaskDelegationEngine(self)

    async def research_task(self, task):
        """Enhanced research with real API integrations"""
        try:
            # Use task delegation for complex tasks
            if self.delegate_tasks.value:
                delegated_results = await self.task_delegation_engine.delegate_tasks(task['task'])
                return self.synthesize_delegated_results(delegated_results)
            else:
                return await super().research_task(task)
        except Exception as e:
            return f"Research completed with some limitations: {str(e)}"

    def synthesize_delegated_results(self, delegated_results):
        """Synthesize results from multiple specialized agents"""
        synthesis = "Comprehensive research synthesis:\n\n"
        for agent_type, results in delegated_results.items():
            synthesis += f"• {agent_type.replace('_', ' ').title()}: {results['results']}\n"
        return synthesis

# Enhanced usage example
enhanced_agent = EnhancedResearchAgent()